The idea with this is just to try and see if there is any thing special with null values in indexes.
I thought i understood it as DB viewing null and like this has no value and therefor if you indexed on a column that is nullable then all the null values would not be index and if you searched for *WHERE name IS NULL* then i thought that it would be a full table scan.
But this might not be the case so i will try to figure that out

In [1]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(20000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (20000 rows)...
Finished inserting 20000 rows into 'products' in 0.35s.


,Tables_in_test_database
0,availability
1,brands
2,categories
3,colors
4,products


Products table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,description,text,YES,,None,
3,brand_id,int,YES,MUL,None,
4,category_id,int,YES,MUL,None,
5,price,float,YES,,None,
6,currency,varchar(10),YES,,None,
7,stock,int,YES,,None,
8,ean,varchar(50),YES,,None,
9,color_id,int,YES,MUL,None,


Brands table


...

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Categories table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Colors table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(100),YES,UNI,None,


So i will try to set some of the fields to null to see what the explain says

In [2]:
query = """
UPDATE products
SET price = NULL
WHERE id <= 1000
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

In [3]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE price IS NULL;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 1000 rows fetched, 5.79 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,20000,10.0,Using where


In [4]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE price = 100;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 27 rows fetched, 5.74 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,20000,10.0,Using where


In [5]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price IS NULL;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 0 rows affected, 21.25 ms
[QUERY METRICS] 1000 rows fetched, 5.14 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_price,idx_price,5,const,1000,100.0,Using index condition


In [6]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

query = """
SELECT * FROM products WHERE price = 100;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 0 rows affected, 19.35 ms
[QUERY METRICS] 27 rows fetched, 0.64 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ref,idx_price,idx_price,5,const,27,100.0,Using index condition


So this seems to me to indicate the null values does get stored in the index

It was because i have read the articles on use the index luke where he talks about the ORACLE DB storign nullable columns as partiel indexes or something. But taht is not the case with mysql